## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Port Elizabeth,ZA,-33.9180,25.5701,69.10,73,0,21.85,clear sky
1,1,Richards Bay,ZA,-28.7830,32.0377,71.33,91,73,13.67,light rain
2,2,Ushuaia,AR,-54.8000,-68.3000,44.26,61,75,9.22,broken clouds
3,3,Lebu,CL,-37.6167,-73.6500,60.17,92,12,7.85,few clouds
4,4,Hermanus,ZA,-34.4187,19.2345,64.90,92,41,5.55,light rain


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
7,7,Mandera,KE,3.9366,41.8670,81.18,32,81,8.66,broken clouds
12,12,Dhidhdhoo,MV,6.8833,73.1000,82.02,76,67,13.33,broken clouds
17,17,Kudahuvadhoo,MV,2.6708,72.8944,82.08,68,97,10.76,overcast clouds
20,20,Monrovia,LR,6.3005,-10.7969,79.14,89,18,6.51,few clouds
32,32,Georgetown,MY,5.4112,100.3354,80.53,92,20,3.44,few clouds
39,39,Prado,BR,-17.3411,-39.2208,78.39,84,43,10.89,scattered clouds
43,43,Sao Gabriel,BR,-30.3364,-54.3200,77.79,31,36,3.49,scattered clouds
45,45,Caravelas,BR,-17.7125,-39.2481,78.04,82,12,8.46,few clouds
48,48,Lamu,KE,-2.2717,40.9020,79.02,88,53,8.70,broken clouds
58,58,Sinnamary,GF,5.3833,-52.9500,75.40,87,66,7.05,light rain


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID        148
City           148
Country        146
Lat            148
Lng            148
Max Temp       148
Humidity       148
Cloudiness     148
Wind Speed     148
Description    148
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID        146
City           146
Country        146
Lat            146
Lng            146
Max Temp       146
Humidity       146
Cloudiness     146
Wind Speed     146
Description    146
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
7,Mandera,KE,81.18,broken clouds,3.9366,41.8670,
12,Dhidhdhoo,MV,82.02,broken clouds,6.8833,73.1000,
17,Kudahuvadhoo,MV,82.08,overcast clouds,2.6708,72.8944,
20,Monrovia,LR,79.14,few clouds,6.3005,-10.7969,
32,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
39,Prado,BR,78.39,scattered clouds,-17.3411,-39.2208,
43,Sao Gabriel,BR,77.79,scattered clouds,-30.3364,-54.3200,
45,Caravelas,BR,78.04,few clouds,-17.7125,-39.2481,
48,Lamu,KE,79.02,broken clouds,-2.2717,40.9020,
58,Sinnamary,GF,75.40,light rain,5.3833,-52.9500,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df['Hotel Name'] == '']
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df.count()

City           131
Country        131
Max Temp       131
Description    131
Lat            131
Lng            131
Hotel Name     131
dtype: int64

In [24]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>weather Description</dt><dd>{Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))